# My implentation of the pagerank

We import libraries

In [1]:
import networkx as nx
import numpy as np
import scipy.sparse as sp
import sys


In [2]:

try:
    import cugraph
    import cudf
    HAS_GPU = True
    print("trovata GPU")
except ImportError:
    HAS_GPU = False
    print("NO GPU")

NO GPU


In [3]:
def build_index_map(G):
    nodes = list(G.nodes())
    node_to_idx = {node: i for i,node in enumerate(nodes)}
    return nodes, node_to_idx

In [4]:
def build_transition_matrix(G, node_to_idx):
    n= len(G)
    row_ind = []
    col_ind = []
    data = []
    dangling_nodes = []
    
    for node in G.nodes():
        u = node_to_idx[node]
        neighbors = list(G.neighbors(node))
        
        if not neighbors:
            #dangling node --> no out-links 
            dangling_nodes.append(u)
        else:
            weight = 1.0 / len(neighbors)
            for neighbor in neighbors:
                v= node_to_idx[neighbor]
                row_ind.append(u)
                col_ind.append(v)
                data.append(weight)
                
    #compressed Sparse row matrix, we use sparse because is more compact
    M= sp.csr_matrix((data,(row_ind, col_ind)),shape=(n,n))
    return M, dangling_nodes

In [5]:
def power_iteration(M, num_nodes, dangling_nodes, alpha=0.85, tol=1.0e-6, max_iter=200):
    #Formula: r_new = alpha * M.T * r_old + (1 - alpha) / N + dangling_correction
    # 1. Initialize PageRank vector uniformly (1/N)
    x = np.ones(num_nodes) / num_nodes
    
    # 2. Pre-calculate the teleportation probability (random jump factor)
    # The probability of jumping to any random node is (1-alpha) / N
    teleport_prob = (1 - alpha) / num_nodes
    
    # Check for convergence
    for i in range(max_iter):
        x_prev = x.copy()
        
        # --- The "Random Walk" Step ---
        
        # Calculate sum of PR mass from dangling nodes
        # If a walker is at a dangling node, they jump to a random node.
        dangling_sum = sum(x_prev[d] for d in dangling_nodes)
        dangling_val = alpha * (dangling_sum / num_nodes)
        
        # Matrix multiplication: Push mass along edges
        # M.T accounts for incoming links (transpose of transition matrix)
        x = alpha * (M.T @ x_prev)
        
        # Add the teleportation factor + dangling node redistribution
        x += teleport_prob + dangling_val
        
        # Check convergence (L1 norm of difference)
        err = np.abs(x - x_prev).sum()
        if err < tol:
            print(f"Converged after {i+1} iterations.")
            return x

    print("Max iterations reached without full convergence.")
    return x

In [6]:
def calculate_pagerank(G, alpha = 0.85):
    
    n= len(G)
    
    nodes, node_to_idx = build_index_map(G)
    
    # 3. Build Transition Matrix (The "Graph Logic")
    print("Building transition matrix...")
    M, dangling_nodes = build_transition_matrix(G, node_to_idx)
    
    # 4. Run Solver (The "Math Logic")
    print("Running PageRank power iteration...")
    scores = power_iteration(M, n, dangling_nodes, alpha=alpha)
    
    # 5. Format Output
    return dict(zip(nodes, scores))